In [ ]:
# 구글 계정 액세스
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 이미지 데이터 train, test, vaild 나누기 

In [ ]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [ ]:
import PIL
import glob
import os, shutil


train_dir = os.path.join('/content/drive/MyDrive/Colab/res50/Plant/train')
valid_dir = os.path.join('/content/drive/MyDrive/Colab/res50/Plant/valid')
test_dir = os.path.join('/content/drive/MyDrive/Colab/res50/Plant/test')

## 모델 구성 ResNet

In [ ]:
# conda install h5py

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

from tensorflow import keras
from tensorflow.keras import optimizers

from tensorflow.keras import Input
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import scipy
import matplotlib.pyplot as plt

In [ ]:
# 입력 데이터 정규화를 위한 클래스 정의
def forward(input):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    mean = mean.reshape(1, 1, 1, 3)
    std = std.reshape(1, 1, 1, 3)
    return (input - mean) / std

In [ ]:
# ResNet50 model 불러오기 
base_model = ResNet50(include_top=False, input_shape =(224, 224 ,3), weights = 'imagenet')
base_model.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [ ]:
# ResNet50 전이학습 
base_model.trainable = False

## 모델 layer 설계
inputs = Input(shape=(None, None, 3))
x = tf.keras.layers.experimental.preprocessing.Resizing(256,256)(inputs)
x = tf.keras.layers.experimental.preprocessing.CenterCrop(224,224)(x)
x = tf.keras.applications.resnet50.preprocess_input(x)
x = base_model(x, training = False)
x = Flatten()(x)   # Fully Connected에 온전하게 학습을 위해 펼쳐준다 

# x = Dense(256, activation = 'relu')(x)
outputs = Dense(5, activation = 'softmax')(x) # Softmax 함수로 5개 분류

model_res = tf.keras.Model(inputs, outputs) # model_res 란 이름의 인풋과 아웃풋이 정해진 모델

In [ ]:
model_res.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resizing (Resizing)         (None, 256, 256, 3)       0         
                                                                 
 center_crop (CenterCrop)    (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                             

In [ ]:
model_res.compile(loss = 'categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),  # RMSprop : 기울기를 단순 누적하지 않고 지수 가중 이동 평균 Exponentially weighted moving average 를 사용하여 최신 기울기들이 더 크게 반영되도록 하였다
             metrics = ['acc'])

In [ ]:
# 모든 이미지를 1/255로 스케일을 조정
train_datagan = ImageDataGenerator(rescale=1./255, preprocessing_function = forward) # forward 정규화 
valid_datagan = ImageDataGenerator(rescale=1./255, preprocessing_function = forward)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

# 이미지 증식 (ImageDataGenerator)
train_datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_datagan = ImageDataGenerator(rescale=1./255, preprocessing_function = forward)

In [ ]:
train_generator = train_datagan.flow_from_directory(
                                            train_dir, 
                                            target_size = (224,224),
                                            batch_size=24,
                                            class_mode='categorical')

Found 1924 images belonging to 5 classes.


In [ ]:
validation_generator = valid_datagan.flow_from_directory(
                                            valid_dir, 
                                            target_size = (224,224),
                                            batch_size=24,
                                            class_mode='categorical')

Found 961 images belonging to 5 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 데이터 크기:', labels_batch.shape)
    break

배치 데이터 크기: (24, 224, 224, 3)
배치 데이터 크기: (24, 5)


In [ ]:
# early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

# 모델 학습 
history = model_res.fit(train_generator,
                    validation_data=validation_generator,
#                    callbacks=[early],
                    epochs=50)


Epoch 1/50
81/81 [==============================] - 1635s 20s/step - loss: 0.6654 - acc: 0.7521 - val_loss: 0.6197 - val_acc: 0.7773
Epoch 2/50
81/81 [==============================] - 15s 181ms/step - loss: 0.2290 - acc: 0.9272 - val_loss: 0.5483 - val_acc: 0.8044
Epoch 3/50
81/81 [==============================] - 15s 180ms/step - loss: 0.1199 - acc: 0.9740 - val_loss: 0.5039 - val_acc: 0.8158
Epoch 4/50
81/81 [==============================] - 15s 181ms/step - loss: 0.0776 - acc: 0.9870 - val_loss: 0.5718 - val_acc: 0.7981
Epoch 5/50
81/81 [==============================] - 15s 180ms/step - loss: 0.0533 - acc: 0.9932 - val_loss: 0.5862 - val_acc: 0.7908
Epoch 6/50
81/81 [==============================] - 15s 181ms/step - loss: 0.0360 - acc: 0.9979 - val_loss: 0.5509 - val_acc: 0.8002
Epoch 7/50
81/81 [==============================] - 15s 179ms/step - loss: 0.0280 - acc: 0.9990 - val_loss: 0.5819 - val_acc: 0.7971
Epoch 8/50
81/81 [==============================] - 15s 180ms/step - 

In [ ]:
# 모델 테스트
test_generator = test_datagan.flow_from_directory(test_dir, target_size=(224,224), batch_size=24, class_mode='categorical')
test_loss, test_acc = model_res.evaluate_generator(test_generator, steps=50)
print(f'test loss: {test_loss}, test acc: {test_acc}')

Found 964 images belonging to 5 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


test loss: 1.1958956718444824, test acc: 0.7416666746139526


In [ ]:
# 모델 저장
saved_model_dir = '/content/drive/MyDrive/Colab/model/resnet50_saved_model1.h5'

model_res.save(saved_model_dir)

In [ ]:
import pickle
import pandas as pd

In [ ]:
from tensorflow.keras.models import load_model

new_model =load_model('/content/drive/MyDrive/Colab/model/resnet50_saved_model1.h5')
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resizing (Resizing)         (None, 256, 256, 3)       0         
                                                                 
 center_crop (CenterCrop)    (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                             

In [ ]:
new_model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001),
                 loss='categorical_crossentropy',
                 metrics=['acc'])
# new_model.evaluate(test_dataset)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc) + 1)

plt.plot(epochs, acc, 'r', label= 'Training acc')
plt.plot(epochs, val_acc, 'b', label= 'Validation acc')
plt.title('TRAINING AND VALIDATION ACCURACY')
plt.legend()

plt.figure()

plt.plot(epochs, loss, label = 'Training loss')
plt.plot(epochs, val_loss, label = 'Validation loss')
plt.title('TRAINING AND VALIDATION LOSS')
plt.legend()

plt.show()

NameError: ignored

In [ ]:
test_generator = test_datagan.flow_from_directory(test_dir,
                                                target_size = (224,224),
                                                batch_size = 24,
                                                class_mode = 'categorical')
test_loss, test_acc = new_model.evaluate_generator(test_generator,steps=50)
print('test acc : ', test_acc)